# test 1 biobert

## imports

In [1]:
from ast import literal_eval
from itertools import chain
from sklearn.metrics import precision_recall_fscore_support
from tqdm.notebook import tqdm, trange
from sklearn.model_selection import StratifiedKFold
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm.autonotebook import tqdm

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow'

In [34]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# load data
import pandas as pd
df = pd.read_csv('../data/raw/mtsamples.csv')
df.transcription=df.transcription.astype(str)
#print(df.columns)
df_test = df.head(15)
df_test.head(2)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."


## cleaning

In [4]:
def convert(lst):
    '''This function converts the keywords to a list of keywords'''
    my_list = lst.split(',')
    print(my_list)
    return my_list
    return ([i for i in lst.split()])
df_test['keywords']=df_test['keywords'].fillna("")
df_test['keywords_list']= df_test['keywords'].apply(lambda x: x.split(','))
#ACHTUNG HIER IST ES IMPORTANT IN EINEM NEXT STEP DIE KEYWORD LIST ZU CLEANEN
df_test.head(2)

/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_1971/1066114346.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['keywords']=df_test['keywords'].fillna("")
/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_1971/1066114346.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['keywords_list']= df_test['keywords'].apply(lambda x: x.split(','))


,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,keywords_list
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller...","[allergy / immunology, allergic rhinitis, al..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh...","[bariatrics, laparoscopic gastric bypass, we..."


In [5]:
def location_indices(stringstext, check_list):
    '''this function finds the location of the keywords in the string'''
    
    res = dict()
    for ele in check_list :
        if ele in stringstext:
            # getting front index
            strt = stringstext.index(ele)
            
            # getting ending index
            res[ele] = [strt, strt + len(ele) - 1]
    return res.values()
        
df_test['location'] = df_test.apply(lambda x: location_indices(x.transcription, x.keywords_list), axis=1) 
df_test.head(2)

/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_1971/4118250391.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['location'] = df_test.apply(lambda x: location_indices(x.transcription, x.keywords_list), axis=1)


,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,keywords_list,location
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller...","[allergy / immunology, allergic rhinitis, al...","([70, 79], [519, 525], [490, 502], [1036, 1044..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh...","[bariatrics, laparoscopic gastric bypass, we...","([1401, 1412], [1386, 1392], [658, 664], [1406..."


In [15]:
#new try Kfold
import numpy as np
frames = []
df_split = np.array_split(df_test, 5)
for split in range(0, 5):
    df_split[split]['kfold'] = split
    frames.append(df_split[split])
dfx = pd.concat(frames)
dfx

#find max len der texte
max_len = df_test['transcription'].map(lambda x: len(x)).max()
max_len
pd.set_option('display.max_colwidth', None)
dfx.head(1)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,keywords_list,location,kfold
0,0,A 23-year-old white female presents with complaint of allergies.,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female presents with complaint of allergies. She used to have allergies when she lived in Seattle but she thinks they are worse here. In the past, she has tried Claritin, and Zyrtec. Both worked for short time but then seemed to lose effectiveness. She has used Allegra also. She used that last summer and she began using it again two weeks ago. It does not appear to be working very well. She has used over-the-counter sprays but no prescription nasal sprays. She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals: Weight was 130 pounds and blood pressure 124/78.,HEENT: Her throat was mildly erythematous without exudate. Nasal mucosa was erythematous and swollen. Only clear drainage was seen. TMs were clear.,Neck: Supple without adenopathy.,Lungs: Clear.,ASSESSMENT:, Allergic rhinitis.,PLAN:,1. She will try Zyrtec instead of Allegra again. Another option will be to use loratadine. She does not think she has prescription coverage so that might be cheaper.,2. Samples of Nasonex two sprays in each nostril given for three weeks. A prescription was written as well.","allergy / immunology, allergic rhinitis, allergies, asthma, nasal sprays, rhinitis, nasal, erythematous, allegra, sprays, allergic,","[allergy / immunology, allergic rhinitis, allergies, asthma, nasal sprays, rhinitis, nasal, erythematous, allegra, sprays, allergic, ]","([70, 79], [519, 525], [490, 502], [1036, 1044], [490, 495], [842, 854], [463, 469], [0, -1])",0


## config 

In [6]:
#configuration

class config:
    MAX_LEN = 42 # 416
    TRAIN_BATCH_SIZE = 8
    VALID_BATCH_SIZE = 8
    EPOCHS = 5
    
    # BERT_PATH = "../input/bert-base-uncased/" 
    # MODEL_PATH = "model.bin"
    # TOKENIZER = tokenizers.BertWordPieceTokenizer(f"{BERT_PATH}/vocab.txt" ,lowercase = True)
    
    BERT_PATH = 'emilyalsentzer/Bio_ClinicalBERT'
    MODEL_PATH = 'emilyalsentzer/Bio_ClinicalBERT'
    TOKENIZER = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    DROPOUT = 0.2
    MAX_GRAD_NORM = 1.0
    LEARNING_RATE = 1e-5

## tokenizer

In [7]:
def loc_list_to_ints(loc_list):
    tuples = []
    for sublist in loc_list:
        tuples.append(tuple(sublist))
    return tuples

In [35]:
def process_data_tokenize(feature_text, annotation, location, tokenizer, max_len):    ##X , Y, selected_text  
    '''this function tokenizes the data'''
    location_list = loc_list_to_ints(location)        
    char_targets = [0] * len(feature_text) #creating empty list(all zeros) of character;it will be made 1 if annotation in text   
    for loc,anno in zip(location_list,annotation): 
        len_st = loc[1] - loc[0]

        idx0 = None
        idx1 = None
        for ind in (i for i, e in enumerate(feature_text) if (e == anno[0] and i == loc[0])):
            if feature_text[ind: ind+len_st] == anno:

                idx0 = ind
                idx1 = ind + len_st - 1
                if idx0 != None and idx1 != None:
                    for ct in range(idx0, idx1 + 1):
                        char_targets[ct] = 1  #replacing zeros with 1 if that part of the text is selected text
        
                break
    print('in process function')
    tokenized_input = tokenizer.encode_plus(feature_text,
                                            add_special_tokens=False, 
                                            return_attention_mask=True, 
                                            return_offsets_mapping=True,
                                            padding =  'max_length',
                                            truncation = True,
                                            max_length=42,
                                            return_tensors = 'pt')
    
       # `encode_plus` will:
       
       #ValueError: 4422 is not a valid PaddingStrategy, please select one of ['longest', 'max_length', 'do_not_pad']
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    
    # for key, value in tokenized_input.items():
    #     print( '{} : {}'.format( key, value ) )
    input_ids = tokenized_input.get('input_ids')
    mask = tokenized_input.get('attention_mask')
    token_type_ids = tokenized_input.get('token_type_ids')
    offsets = tokenized_input.get('offset_mapping')
    #print(feature_text)
    # print('token_type_ids.shape', token_type_ids.shape)
    print('offsets.shape', offsets.shape)
    
    
    target_idx = []
    for value in offsets:
        for j, (offset1, offset2) in enumerate(value):
            #print(j, offset1, offset2)
            if sum(char_targets[offset1: offset2]) > 0:
                target_idx.append(j)

    #creating label
    ignore_idxes = np.where(np.array(token_type_ids) != 1)[0]
    label = np.zeros(offsets.size())#, dtype=np.float32)]))
    label[ignore_idxes] = -1
    label[target_idx] = 1

    
    return {
    'ids': input_ids,
    'mask': mask,
    'token_type_ids': token_type_ids,
    'labels': label,
    'offsets': offsets
}

In [175]:
import numpy as np
output = process_data_tokenize(example["feature_text"],example["annotation"],example["location"],config.TOKENIZER,config.MAX_LEN)

#for key in output.keys():
#    print(key)
#    print(output[key])
#    print("=" * 100)
    
output


df_test.head(1)
df_traindict = {
    "feature_text": df_test.transcription,
    "location": df_test.location,
    "annotation": df_test.keywords_list
}
#list(df_traindict['feature_text'])
output_train = process_data_tokenize(list(df_traindict["feature_text"]),list(df_traindict["annotation"]),list(df_traindict["location"]),config.TOKENIZER,config.MAX_LEN)
#output_train = process_data_tokenize(df_traindict["feature_text"],df_traindict["annotation"],df_traindict["location"],config.TOKENIZER,config.MAX_LEN)
output_train

process_data_tokenize
annp ['cardiovascular', '/', 'pulmonary,', '2-d', 'm-mode,', 'doppler,', 'aortic', 'valve,', 'atrial', 'enlargement,', 'diastolic', 'function,', 'ejection', 'fraction,', 'mitral,', 'mitral', 'valve,', 'pericardial', 'effusion,', 'pulmonary', 'valve,', 'regurgitation,', 'systolic', 'function,', 'tricuspid,', 'tricuspid', 'valve,', 'normal', 'lv']
*************
loc dict_values([[290, 295], [297, 302], [24, 29], [216, 224], [176, 183], [304, 309], [243, 253], [339, 347], [136, 143], [318, 326], [299, 300]])
*************
label [[[0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.

TypeError: 'dict_values' object is not subscriptable

## Data Loader

In [9]:
#Data loader
class NBMEDataset:
    def __init__(self,  feature_text, annotation, location):   #text(X) #label(Y), #selected_text #start:end
        #self.pn_history = pn_history
        self.feature_text = list(feature_text)
        self.annotation = list(annotation)
        self.location = list(location)
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
        
    def __len__(self):
        return len(self.feature_text)
        
    def __getitem__(self, item):
        data = process_data_tokenize(
            #self.pn_history[item],
            self.feature_text[item],
            self.annotation[item],
            self.location[item],
            self.tokenizer,
            self.max_len
        )
        return {
            'ids': torch.tensor(data["ids"]), #input_ids
            'mask': torch.tensor(data["mask"], dtype=torch.long), #attention_mask
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long), #segment_ids
            'labels': torch.tensor(data["labels"], dtype=torch.long), 
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }
        

## Model

In [29]:
import transformers
class NBMEModel(transformers.BertPreTrainedModel):    #torch.nn.Module
    def __init__(self,conf):
        super(NBMEModel,self).__init__(conf)
        self.bert = transformers.BertModel.from_pretrained(config.BERT_PATH, config=conf)
        self.dropout = torch.nn.Dropout(config.DROPOUT)
        self.classifier = torch.nn.Linear(768, 1)
        torch.nn.init.normal_(self.classifier.weight, std=0.02) 
        
    def forward(self, ids, mask, token_type_ids):
        sequence_out = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)[0] #last_hidden_state
        batch_size, max_len, feat_dim = sequence_out.shape
        sequence_output = self.dropout(sequence_out)
        logits = self.classifier(sequence_output)
        logits = logits.squeeze(-1) 
        return logits

In [11]:
#utility function
class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
#loss function
def loss_fn(logits, labels):
    print('in loss function')
    #labels = labels.unsqueeze(1)
    loss_fct = torch.nn.BCEWithLogitsLoss(reduction = "none")
    loss = loss_fct(logits,labels)
    return loss

## training function

In [39]:
# in a next step we use data loader to load the data - https://blog.paperspace.com/dataloaders-abstractions-pytorch/
# in a next step with the help of the optimizer we will update the weights of the model
#training function


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def train_fn(dataloader, model, optimizer, scheduler=None): #before: dataloader
    model.train()
    losses = AverageMeter() 
    print('in function train_fn')

    for batch_idx, batch in enumerate(tqdm(dataloader, total=len(dataloader))):
        for k, v in batch.items():
            batch[k] = v.to(DEVICE)

        optimizer.zero_grad()
        
        ids = batch['ids'].squeeze(1)
        print('batchids',ids.shape)

        #output = torch.squeeze(input)
        print('batchidssqueeze',ids.shape)
        logits = model(ids=ids, mask=batch['mask'], token_type_ids=batch['token_type_ids'])
        #model(ids=batch['ids'], mask=batch['mask'], token_type_ids=batch['token_type_ids'])
        loss = loss_fn(logits,batch["labels"])
        loss = loss.mean()
        # loss = torch.masked_select(loss, labels > -1.0).mean()
        # losses.update(loss.item(),ids.size(0))
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        losses.update(loss.item(), batch['ids'].size(0))
        dataloader.set_postfix(loss=losses.avg) #tk. bfefore
        #or like this - https://www.kaggle.com/code/daisybbb/pytorch-pubmedbert-infer


        torch.nn.utils.clip_grad_norm_(model.parameters(), config.MAX_GRAD_NORM)
        optimizer.step()
        scheduler.step() ## Update learning rate schedule
    
    # #output = torch.argmax(torch.softmax(logits, dim=2),dim=2).cpu().detach().numpy()
        tqdm(dataloader, total=len(dataloader)).set_postfix(loss=losses.avg)
        
    return losses.avg


In [34]:
 # train model
 #adam w - https://medium.com/@nishantnikhil/adam-optimizer-notes-ddac4fd7218
 #other optimizer https://medium.com/geekculture/a-2021-guide-to-improving-cnns-optimizers-adam-vs-sgd-495848ac6008
 #https://firiuza.medium.com/optimizers-for-training-neural-networks-e0196662e21e
 #https://medium.com/@ThiyaneshwaranG/types-of-optimizers-in-deep-learning-from-gradient-descent-to-adam-1572e657c8c5
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

#-------------------
df_test.head(1)
df_traindict = {
    "feature_text": df_test.transcription,
    "location": df_test.location,
    "annotation": df_test.keywords_list
}
list(df_traindict['feature_text'])
output_train = process_data_tokenize(list(df_traindict["feature_text"]),list(df_traindict["annotation"]),list(df_traindict["location"]),config.TOKENIZER,config.MAX_LEN)
#output_train = process_data_tokenize(df_traindict["feature_text"],df_traindict["annotation"],df_traindict["location"],config.TOKENIZER,config.MAX_LEN)
output_train



#-----------------#
model_config = transformers.BertConfig.from_pretrained(config.BERT_PATH)
model_config.output_hidden_states = True
model = NBMEModel(conf=model_config)
model.to(DEVICE)
    
#num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
param_optimizer = list(model.named_parameters())
#print('param_optimizer', param_optimizer)
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]

optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
optimizer = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=2
    )
#-----------------#
train_loss = train_fn(x, model, optimizer, scheduler=scheduler)
#train_loss_data.append(train_loss)
print(f"Train loss: {train_loss}")

TypeError: 'dict_values' object is not subscriptable

## evaluation function

In [ ]:
#evaluation function
def eval_fn(dataloader, model): #before: dataloader
    model.eval()
    losses = AverageMeter() # Computes and stores the average and current value

    with torch.no_grad():
        tk = tqdm(dataloader, total=len(dataloader)) 
        #tk = tqdm(data, total=5)
        
        # ids = data['ids']
        # token_type_ids = data["token_type_ids"]
        # mask = data["mask"]
        # labels = data["labels"]
        # offsets = data["offsets"]#    
        # labels = labels.unsqueeze(0)

        for batch, data in enumerate(tk):

            ids = data['ids']
            token_type_ids = data["token_type_ids"]
            mask = data["mask"]
            labels = data["labels"]
            offsets = data["offsets"]

            ids = ids.to(DEVICE, dtype=torch.long)
            token_type_ids = token_type_ids.to(DEVICE, dtype=torch.long)
            mask = mask.to(DEVICE, dtype=torch.long)
            labels = labels.to(DEVICE, dtype=torch.float64)

            logits = model(ids=ids, mask=mask, token_type_ids=token_type_ids) #last_hidden_state
            
            loss = loss_fn(logits, labels)
            loss = torch.masked_select(loss, labels > -1.0).mean()
            losses.update(loss.item(),ids.size(0))
            tk.set_postfix(loss=losses.avg)
        
        return losses.avg

In [ ]:
df_test.head(1)
df_traindict = {
    "feature_text": df_test.transcription,
    "location": df_test.location,
    "annotation": df_test.keywords_list
}
list(df_traindict['feature_text'])
output_train = process_data_tokenize(list(df_traindict["feature_text"]),list(df_traindict["annotation"]),list(df_traindict["location"]),config.TOKENIZER,config.MAX_LEN)
#output_train = process_data_tokenize(df_traindict["feature_text"],df_traindict["annotation"],df_traindict["location"],config.TOKENIZER,config.MAX_LEN)
output_train

TypeError: 'dict_values' object is not subscriptable

## Run training

In [40]:

#training
import time
import numpy as np
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
def run(fold):
    
    train_loss_data, valid_loss_data = [], []
    best_loss = np.inf
    since = time.time()
   
    df_train = dfx[dfx.kfold != fold].reset_index(drop=True) 
    df_valid = dfx[dfx.kfold == fold].reset_index(drop=True)
    
    train_dataset = NBMEDataset(
        feature_text=df_train.transcription.values,
        annotation=df_train.keywords_list.values,
        location=df_train.location.values
        
    )
    
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=0
    )

    valid_dataset = NBMEDataset(
        feature_text=df_valid.transcription.values,
        annotation=df_valid.keywords_list.values,
        location=df_valid.location.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=0
    )

    model_config = transformers.BertConfig.from_pretrained(config.BERT_PATH)
    model_config.output_hidden_states = True
    model = NBMEModel(conf=model_config)
    model.to(DEVICE)
    
    num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=config.LEARNING_RATE)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )

    best_loss = np.inf
    
    for i in range(config.EPOCHS):
        print("Epoch: {}/{}".format(i + 1, config.EPOCHS))

        # train model
        train_loss = train_fn(train_data_loader
                              , model, optimizer, scheduler=scheduler)
        train_loss_data.append(train_loss)
        print(f"Train loss: {train_loss}")

        # evaluate model - not now
      #  valid_loss = eval_fn(valid_data_loader, model)
      #  valid_loss_data.append(valid_loss)
      #  print(f"Valid loss: {valid_loss}")


       # if valid_loss < best_loss:
        #    best_loss = valid_loss
        torch.save(model.state_dict(), "model_fold1.bin")


        #time_elapsed = time.time() - since
        #print('Training completed in {:.0f}m {:.0f}s'.format(
        #    time_elapsed // 60, time_elapsed % 60))
    
    
run(fold=0)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 1/5
in function train_fn


  0%|          | 0/2 [00:00<?, ?it/s]

/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_1971/1640454316.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'ids': torch.tensor(data["ids"]), #input_ids
/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_1971/1640454316.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'mask': torch.tensor(data["mask"], dtype=torch.long), #attention_mask
/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_1971/1640454316.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids': torch.tensor(data["token_ty

in process function
offsets.shape torch.Size([1, 42, 2])
in process function
offsets.shape torch.Size([1, 42, 2])
in process function
offsets.shape torch.Size([1, 42, 2])
in process function
offsets.shape torch.Size([1, 42, 2])
in process function
offsets.shape torch.Size([1, 42, 2])
in process function
offsets.shape torch.Size([1, 42, 2])
in process function
offsets.shape torch.Size([1, 42, 2])
in process function
offsets.shape torch.Size([1, 42, 2])
batchids torch.Size([8, 42])
batchidssqueeze torch.Size([8, 42])


RuntimeError: number of dims don't match in permute

# test 2 keyword berd

In [ ]:
https://medium.com/geekculture/3-techniques-to-perform-named-entity-recognition-ner-on-text-data-ec1e91e3a8aa
from sklearn.feature_extraction.text import CountVectorizer

doc = """
         Supervised learning is the machine learning task of 
         learning a function that maps an input to an output based 
         on example input-output pairs.[1] It infers a function 
         from labeled training data consisting of a set of 
         training examples.[2] In supervised learning, each 
         example is a pair consisting of an input object 
         (typically a vector) and a desired output value (also 
         called the supervisory signal). A supervised learning 
         algorithm analyzes the training data and produces an 
         inferred function, which can be used for mapping new 
         examples. An optimal scenario will allow for the algorithm 
         to correctly determine the class labels for unseen 
         instances. This requires the learning algorithm to  
         generalize from the training data to unseen situations 
         in a 'reasonable' way (see inductive bias).
      """


n_gram_range = (1, 3)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names()
candidates

/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['algorithm',
 'algorithm analyzes',
 'algorithm analyzes training',
 'algorithm correctly',
 'algorithm correctly determine',
 'algorithm generalize',
 'algorithm generalize training',
 'allow',
 'allow algorithm',
 'allow algorithm correctly',
 'analyzes',
 'analyzes training',
 'analyzes training data',
 'based',
 'based example',
 'based example input',
 'bias',
 'called',
 'called supervisory',
 'called supervisory signal',
 'class',
 'class labels',
 'class labels unseen',
 'consisting',
 'consisting input',
 'consisting input object',
 'consisting set',
 'consisting set training',
 'correctly',
 'correctly determine',
 'correctly determine class',
 'data',
 'data consisting',
 'data consisting set',
 'data produces',
 'data produces inferred',
 'data unseen',
 'data unseen situations',
 'desired',
 'desired output',
 'desired output value',
 'determine',
 'determine class',
 'determine class labels',
 'example',
 'example input',
 'example input output',
 'example pair',
 'examp

In [ ]:
#Next, we convert both the document as well as the candidate keywords/keyphrases to numerical data.
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)
candidate_embeddings


array([[-1.2811013 , -0.19414134,  0.18961014, ..., -0.6902669 ,
        -0.08735337, -0.05631514],
       [-0.98599565, -0.3114873 ,  0.11392305, ..., -0.5420821 ,
         0.03869215,  0.04444062],
       [-0.7794865 , -0.3172609 ,  0.30012545, ..., -0.44612113,
         0.09859169,  0.41761875],
       ...,
       [-0.35578147,  0.01782149,  0.374094  , ..., -0.1977843 ,
         0.0460116 , -0.29936978],
       [-0.7603124 , -0.5274268 ,  0.29681763, ..., -0.3508133 ,
         0.16764224, -0.566111  ],
       [-0.6121529 , -0.44180587, -0.27497974, ..., -0.31791598,
         0.22374944, -0.78795713]], dtype=float32)

We are Distilbert as it has shown great performance in similarity tasks, which is what we are aiming for with keyword/keyphrase extraction!

Since transformer models have a token limit, you might run into some errors when inputting large documents. In that case, you could consider splitting up your document into paragraphs and mean pooling (taking the average of) the resulting vectors.

In the final step, we want to find the candidates that are most similar to the document. We assume that the most similar candidates to the document are good keywords/keyphrases for representing the document.

To calculate the similarity between candidates and the document, we will be using the cosine similarity between vectors as it performs quite well in high-dimensionality:

In [ ]:

from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keywords

['algorithm analyzes training',
 'learning algorithm generalize',
 'learning machine learning',
 'learning algorithm analyzes',
 'algorithm generalize training']

There is a reason why similar results are returned… they best represent the document! If we were to diversify the keywords/keyphrases then they are less likely to represent the document well as a collective.

Thus, the diversification of our results requires a delicate balance between the accuracy of keywords/keyphrases and the diversity between them.

There are two algorithms that we will be using to diversify our results:

Max Sum Similarity
Maximal Marginal Relevance
Max Sum Similarity
The maximum sum distance between pairs of data is defined as the pairs of data for which the distance between them is maximized. In our case, we want to maximize the candidate similarity to the document whilst minimizing the similarity between candidates.

To do this, we select the top 20 keywords/keyphrases, and from those 20, select the 5 that are the least similar to each other:
https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea

In [ ]:
import numpy as np
import itertools

def max_sum_sim(doc_embedding, word_embeddings, words, top_n, nr_candidates):
    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['maps', 'input', 'class', 'training', 'algorithm']

Maximal Marginal Relevance
The final method for diversifying our results is Maximal Marginal Relevance (MMR). MMR tries to minimize redundancy and maximize the diversity of results in text summarization tasks. Fortunately, a keyword extraction algorithm called EmbedRank has implemented a version of MMR that allows us to use it for diversifying our keywords/keyphrases.

We start by selecting the keyword/keyphrase that is the most similar to the document. Then, we iteratively select new candidates that are both similar to the document and not similar to the already selected keywords/keyphrases:

In [ ]:

import numpy as np

def mmr(doc_embedding, word_embeddings, words, top_n, diversity):

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['learning', 'algorithm', 'training', 'class', 'mapping']

In [ ]:
https://github.com/MaartenGr/KeyBERT/blob/master/keybert/_model.py
https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea

In [ ]:
https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea

from transformers import DistilBertModel
import torch
import torch.nn as nn
class DistilBERT_Model(nn.Module):
 def __init__(self, classes):
   super(DistilBERT_Model, self).__init__()
   self.distilbert = DistilBertModel.from_pretrained('distilbert
                                                     base-uncased')
   self.out = nn.Linear(self.distilbert.config.hidden_size, classes)
   self.sigmoid = nn.Sigmoid()
 def forward(self, input, attention_mask):
   _, output = self.distilbert(input, attention_mask 
                                      = attention_mask)
   out = self.sigmoid(self.out(output))
   return out